In [1]:
import string
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
df_fake = pd.read_csv('datasets/Fake.csv')
df_real = pd.read_csv('datasets/True.csv')

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df_fake['class'] = 0
df_real['class'] = 1

In [6]:
df_fake.shape, df_real.shape

((23481, 5), (21417, 5))

In [7]:
def split_manual_testing(num=10):
    global df_fake
    global df_real
    # Lấy ra số lượng sample tương ứng với số {num} để làm test thủ công
    df_fake_manual_testing = df_fake.tail(num)
    df_real_manual_testing = df_real.tail(num)
    
    start = df_fake.shape[0] - 1
    for idx in range(start, start - num, -1):
        df_fake.drop([idx], axis=0, inplace=True)
    
    start = df_real.shape[0] - 1
    for idx in range(start, start - num, -1):
        df_real.drop([idx], axis=0, inplace=True)
    
    return df_fake_manual_testing, df_real_manual_testing

In [8]:
df_manual_testing = pd.concat(split_manual_testing(), axis=0)

In [9]:
df_manual_testing.to_csv("datasets/manual_testing.csv")

In [10]:
df_merge = pd.concat([df_real, df_fake], axis=0)

In [11]:
df_merge.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [12]:
df = df_merge.drop(['title', 'subject', 'date'], axis=1)
df.head()

,text,class
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [13]:
df = df.sample(frac=1)

In [14]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [15]:
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W]', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', ' ', text)
    return text

In [16]:
df['text'] = df['text'].apply(word_drop)

In [17]:
df.head()

,text,class
15243,amsterdam reuters the international criminal ...,1
12522,are the most criminal brazen and crooked group...,0
5907,by nearly all accounts donald trump s response...,0
17280,here s one sure way to turn a solidly red st...,0
14625,nah nah nah nah hey hey hey good bye support f...,0


In [18]:
x = df['text']
y = df['class']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

### Apply Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
lr = LogisticRegression()
lr.fit(xv_train, y_train)

LogisticRegression()

In [24]:
lr.score(xv_test, y_test)

0.9869652406417112

In [25]:
pred_lr = lr.predict(xv_test)

In [26]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4732
           1       0.99      0.99      0.99      4244

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



### Aplly Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier

In [28]:
dt = DecisionTreeClassifier()
dt.fit(xv_train, y_train)

DecisionTreeClassifier()

In [29]:
dt.score(xv_test, y_test)

0.9959893048128342

In [30]:
pred_dt = dt.predict(xv_test)

In [31]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4732
           1       1.00      1.00      1.00      4244

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



### Apply Gradient Boosting

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

In [33]:
gb = GradientBoostingClassifier()
gb.fit(xv_train, y_train)

GradientBoostingClassifier()

In [34]:
gb.score(xv_test, y_test)

0.9954322638146168

In [35]:
pred_gb = gb.predict(xv_test)

In [36]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4732
           1       0.99      1.00      1.00      4244

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



### Apply Radom Forest

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
rdf = RandomForestClassifier()
rdf.fit(xv_train, y_train)

RandomForestClassifier()

In [39]:
rdf.score(xv_test, y_test)

0.989193404634581

In [40]:
pred_rdf = rdf.predict(xv_test)

In [41]:
print(classification_report(y_test, pred_rdf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4732
           1       0.99      0.99      0.99      4244

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



### Manual Testing

In [42]:
def label(n):
    return "Fake News" if n == 0 else "Not a Fake News"

In [69]:
def manual_testing(news):
    testing_news = {'text':[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test['text'].apply(word_drop)
    new_x_test = new_def_test['text']
    new_xv_test = vectorization.transform(new_x_test)
    pred_lr = lr.predict(new_xv_test)
    pred_dt = dt.predict(new_xv_test)
    pred_gb = gb.predict(new_xv_test)
    pred_rdf = rdf.predict(new_xv_test)
    
    print(f"Logistic Regression Prediction: {label(pred_lr):>20}"
          f"\nDecision Tree Prediction: {label(pred_dt):>26}"
          f"\nGradient Boosting Prediction: {label(pred_gb):>22}"
          f"\nRandom Forest Prediction: {label(pred_rdf):>26}")

In [70]:
news = str(input("Enter the news you want to test: "))
manual_testing(news)

Logistic Regression Prediction:      Not a Fake News
Decision Tree Prediction:            Not a Fake News
Gradient Boosting Prediction:        Not a Fake News
Random Forest Prediction:            Not a Fake News
